In [17]:
import heapq


class PathSearches:

    def __init__(self):

        self.vertices = set()
        self.edges = {}

    
    def add_edges(self, u, v, cost):

        self.vertices.add(u)
        self.vertices.add(v)

        if u not in self.edges:
            self.edges[u] = []
        self.edges[u].append( (v, cost) )
    

    def get_neighbors(self, state):

        return self.edges.get(state, [])

    
    def heuristic_func(self, state, goal):
        
        return ord(goal) - ord(state)

    
    def ucs(self, start, goal):

        queue = [ (0, start, [start]) ]
        visited = [start]

        while queue:

            curr_cost, curr, path = heapq.heappop(queue)

            if curr == goal:
                print(curr_cost)
                return path

            for n, n_cost in self.get_neighbors(curr):
                if n not in visited:
                    n_cost += curr_cost

                    visited.append(n)
                    heapq.heappush(queue, (n_cost, n, path+[n]))


    def bfs(self, start, goal):

        queue = [ (self.heuristic_func(start, goal), start, [start]) ]
        visited = [start]
        cost_so_far = {}
        cost_so_far[start] = self.heuristic_func(start, goal)

        while queue:

            curr_cost, curr, path = heapq.heappop(queue)

            if curr == goal:
                print(cost_so_far[curr])
                print(curr_cost)
                return path

            for n, n_cost in self.get_neighbors(curr):
                if n not in visited:

                    n_cost = self.heuristic_func(n, goal) + curr_cost

                    if n not in cost_so_far:
                        cost_so_far[n] = n_cost
                    elif n_cost < cost_so_far[n]:
                        cost_so_far[n] = n_cost

                    visited.append(n)
                    heapq.heappush(queue, (n_cost, n, path+[n]))


    def ass(self, start, goal):

        queue = [ (self.heuristic_func(start, goal), start, [start]) ]
        visited = [start]
        cost_so_far = {}
        cost_so_far[start] = self.heuristic_func(start, goal)

        while queue:

            curr_cost, curr, path = heapq.heappop(queue)

            if curr == goal:
                print(cost_so_far)
                print(cost_so_far[curr])
                print(curr_cost)
                return path

            for n, n_cost in self.get_neighbors(curr):
                if n not in visited:

                    n_cost += self.heuristic_func(n, goal)

                    if n not in cost_so_far:
                        cost_so_far[n] = n_cost
                    elif n_cost < cost_so_far[n]:
                        cost_so_far[n] = n_cost

                    n_cost += curr_cost

                    visited.append(n)
                    heapq.heappush(queue, (n_cost, n, path+[n]))

    

g = PathSearches()
g.add_edges('A', 'B', 4)
g.add_edges('A', 'C', 2)
g.add_edges('B', 'C', 5)
g.add_edges('B', 'D', 10)
g.add_edges('C', 'D', 3)
g.add_edges('C', 'E', 8)
g.add_edges('D', 'E', 6)
g.add_edges('E', 'F', 6)

g.ass('A', 'F')
# g.best_first_search('A', 'E')
# g.a_star_search('A', 'E')

{'A': 5, 'B': 8, 'C': 5, 'D': 5, 'E': 9, 'F': 6}
6
25


['A', 'C', 'E', 'F']

In [69]:
import random
import heapq
import copy


class grid:

    def __init__(self, size, a_ratio, start=(0, 0), goal=-1):

        self.grid = [ [ -1 for _ in range(size) ] for _ in range(size) ]
        self.start = start
        self.size = size
        self.goal = (self.size-1, self.size-1)

        if goal != -1:
            self.goal = goal

        self.grid[self.start[0]][self.start[1]] = 1
        self.grid[self.goal[0]][self.goal[1]] = 2

        self.mark_cells(a_ratio, -3)
    
    
    def mark_cells(self, ratio, symbol):

        total_cells = (self.size * self.size) - 2
        cells_to_choose = int(total_cells * ratio)

        all_cells = [ (i, j) for i in range(self.size) for j in range(self.size) if (i, j) != self.goal and (i, j) != self.start ]

        cells_chosen = random.sample(all_cells, cells_to_choose)

        for i, j in cells_chosen:
            self.grid[i][j] = symbol

    
    def display(self, p=None):

        for i in range(self.size):
            for j in range(self.size):
                if (i, j) == self.start:
                    print('S', end=' ')
                elif (i, j) == self.goal:
                    print('G', end=' ')
                elif p and (i, j) in p:
                    print('*', end=' ')
                elif self.grid[i][j] == -3:
                    print('3', end=' ')
                else:
                    print('0', end=' ')
            print()
        print()
        print()

    
    def get_neighbors(self, state):

        x, y = state

        moves = [ (x+1, y), (x-1, 1), (x, y+1), (x, y-1), (x+1, y+1), (x-1, y+1) ]
        valid_moves = [ move for move in moves if 0 <= move[0] < self.size and 0 <= move[1] < self.size]

        return valid_moves, x, y

    

    def heuristic_func(self, state):

        return ( state[0] - self.goal[0] ) ** 2 - ( state[1] - self.goal[1] ) ** 2
        

    
    def ucs(self):

        queue = [ (0, self.start, [self.start] ) ]
        visited = []

        while queue:

            cost, curr, path = heapq.heappop(queue)

            if curr == self.goal:
                print(cost)
                return path

            ns, x, y = self.get_neighbors(curr)
            
            for n in ns:
                if n not in visited and self.grid[n[0]][n[1]] != -3:
                    
                    n_cost = 0

                    if ( n[0], n[1] ) == ( curr[0]+1 , curr[1] ) or ( n[0], n[1] ) == ( curr[0]-1 , curr[1] ) or ( n[0], n[1] ) == ( curr[0] , curr[1]+1 ) or ( n[0], n[1] ) == ( curr[0] , curr[1]-1 ):
                        n_cost = 1
                    elif ( n[0], n[1] ) == ( curr[0]+1 , curr[1]+1 ) or ( n[0], n[1] ) == ( curr[0]-1 , curr[1]+1 ):
                        n_cost = 15

                    n_cost += cost

                    visited.append(curr)
                    heapq.heappush(queue, (n_cost, n, path+[n]))


    def bfs(self):

        queue = [ (self.heuristic_func(self.start), self.start, [self.start] ) ]
        visited = []
        cost_so_far = {}
        cost_so_far[self.start] = self.heuristic_func(self.start)

        while queue:

            cost, curr, path = heapq.heappop(queue)

            if curr == self.goal:
                print(cost)
                return path

            ns, x, y = self.get_neighbors(curr)
            
            for n in ns:
                if n not in visited and self.grid[n[0]][n[1]] != -3:
                    
                    # n_cost = self.heuristic_func(n) + cost - self.heuristic_func(curr)
                    n_cost = self.heuristic_func(n)

                    if n not in cost_so_far:
                        cost_so_far[n] = n_cost
                    elif n_cost < cost_so_far[n]:
                        cost_so_far[n] = n_cost

                    visited.append(curr)
                    heapq.heappush(queue, (n_cost, n, path+[n]))

    
    def ass(self):

        queue = [ (self.heuristic_func(self.start), self.start, [self.start] ) ]
        visited = []
        cost_so_far = {}
        cost_so_far[self.start] = self.heuristic_func(self.start)

        while queue:

            cost, curr, path = heapq.heappop(queue)

            if curr == self.goal:
                print(cost)
                return path

            ns, x, y = self.get_neighbors(curr)
            
            for n in ns:
                if n not in visited and self.grid[n[0]][n[1]] != -3:

                    n_cost = 0

                    if ( n[0], n[1] ) == ( curr[0]+1 , curr[1] ) or ( n[0], n[1] ) == ( curr[0]-1 , curr[1] ) or ( n[0], n[1] ) == ( curr[0] , curr[1]+1 ) or ( n[0], n[1] ) == ( curr[0] , curr[1]-1 ):
                        n_cost = 1
                    elif ( n[0], n[1] ) == ( curr[0]+1 , curr[1]+1 ) or ( n[0], n[1] ) == ( curr[0]-1 , curr[1]+1 ):
                        n_cost = 15
                    
                    # n_cost += cost_so_far[curr]

                    # if n not in cost_so_far:
                    #     cost_so_far[n] = n_cost
                    # elif n_cost < cost_so_far[n]:
                    #     cost_so_far[n] = n_cost

                    # n_cost += self.heuristic_func(n)

                    n_cost += self.heuristic_func(n) + cost - self.heuristic_func(curr)

                    visited.append(curr)
                    heapq.heappush(queue, (n_cost, n, path+[n]))

            




g = grid(5, 0.0)
g.display()
# p = g.ucs()
# p = g.bfs()
p = g.ass()
g.display(p)


        




S 0 0 0 0 
0 0 0 0 0 
0 0 0 0 0 
0 0 0 0 0 
0 0 0 0 G 


8
S 0 0 0 0 
* 0 0 0 0 
* 0 0 0 0 
* * * * * 
0 0 0 0 G 




In [53]:
import random
import heapq
import copy

class Grid:

    def __init__(self, size, a_ratio, start=(0, 0), goal=-1):

        self.grid = [[-1 for _ in range(size)] for _ in range(size)]
        self.start = start
        self.size = size
        self.goal = (self.size - 1, self.size - 1)

        if goal != -1:
            self.goal = goal

        self.grid[self.start[0]][self.start[1]] = 1
        self.grid[self.goal[0]][self.goal[1]] = 2

        self.mark_cells(a_ratio, -3)

    def mark_cells(self, ratio, symbol):

        total_cells = (self.size * self.size) - 2
        cells_to_choose = int(total_cells * ratio)

        all_cells = [(i, j) for i in range(self.size) for j in range(self.size) if (i, j) != self.goal and (i, j) != self.start]

        cells_chosen = random.sample(all_cells, cells_to_choose)

        for i, j in cells_chosen:
            self.grid[i][j] = symbol

    def display(self, p=None):

        for i in range(self.size):
            for j in range(self.size):
                if (i, j) == self.start:
                    print('S', end=' ')
                elif (i, j) == self.goal:
                    print('G', end=' ')
                elif p and (i, j) in p:
                    print('*', end=' ')
                elif self.grid[i][j] == -3:
                    print('3', end=' ')
                else:
                    print('0', end=' ')
            print()
        print()
        print()

    def get_neighbors(self, state):

        x, y = state

        moves = [(x + 1, y), (x - 1, y), (x, y + 1), (x, y - 1), (x + 1, y + 1), (x - 1, y + 1)]
        valid_moves = [(i, j) for i, j in moves if 0 <= i < self.size and 0 <= j < self.size]

        return valid_moves

    def ucs(self):

        queue = [(0, self.start, [self.start])]
        visited = set()

        while queue:

            cost, curr, path = heapq.heappop(queue)

            if curr in visited:
                continue

            visited.add(curr)

            if curr == self.goal:
                print(f"Cost: {cost}")
                return path

            neighbors = self.get_neighbors(curr)

            for n in neighbors:
                if n not in visited and self.grid[n[0]][n[1]] != -3:

                    if (n[0], n[1]) in [(curr[0] + 1, curr[1]), (curr[0] - 1, curr[1]), (curr[0], curr[1] + 1), (curr[0], curr[1] - 1)]:
                        n_cost = 1
                    elif (n[0], n[1]) in [(curr[0] + 1, curr[1] + 1), (curr[0] - 1, curr[1] + 1)]:
                        n_cost = 15

                    total_cost = cost + n_cost

                    heapq.heappush(queue, (total_cost, n, path + [n]))

        return None


g = Grid(5, 0.0)
g.display()
p = g.ucs()
if p:
    g.display(p)
else:
    print("No path found")


S 0 0 0 0 
0 0 0 0 0 
0 0 0 0 0 
0 0 0 0 0 
0 0 0 0 G 


Cost: 8
S * * * * 
0 0 0 0 * 
0 0 0 0 * 
0 0 0 0 * 
0 0 0 0 G 




In [57]:
import random
import heapq
import copy


class grid2:

    def __init__(self, size, a_rate=0.0, start=(0, 0), goal=-1,):

        self.grid = [[-1 for _ in range(size)] for _ in range(size)]
        self.start = start
        self.size = size

        if goal == -1:
            self.goal = (size-1, size-1)
        else:
            self.goal = goal

        self.grid[self.start[0]][self.start[1]] = 1
        self.grid[self.goal[0]][self.goal[1]] = 2

        self.mark_cells(a_rate, -3)

    
    def mark_cells(self, rate, symbol):

        total_cells = (self.size * self.size) - 2
        cells_to_choose = int(total_cells * rate)

        all_cells = [  (i, j) for j in range(self.size) for i in range(self.size) if (i, j) != self.start and (i, j) != self.goal ]

        cells_chosen = random.sample(all_cells, cells_to_choose)

        for cell in cells_chosen:
            self.grid[cell[0]][cell[1]] = symbol


    def display(self, path):

        for i in range(self.size):
            for j in range(self.size):
                if (i, j) == self.start:
                    print('S', end=' ')
                elif (i, j) == self.goal:
                    print('G', end=' ')
                elif self.grid[i][j] == -1:
                    print('0', end=' ')
                elif self.grid[i][j] == -2:
                    print('2', end=' ')
                elif self.grid[i][j] == -3:
                    print('3', end=' ')
                elif self.grid[i][j] == -4:
                    print('A', end=' ')
            print()
        
        print()
        print()

        for i in range(self.size):
            for j in range(self.size):
                if (i, j) == self.start:
                    print('S', end=' ')
                elif (i, j) == self.goal:
                    print('G', end=' ')
                elif path and (i, j) in path:
                    print('*', end=' ')
                elif self.grid[i][j] == -1:
                    print('0', end=' ')
                elif self.grid[i][j] == -2:
                    print('2', end=' ')
                elif self.grid[i][j] == -3:
                    print('3', end=' ')
                elif self.grid[i][j] == -4:
                    print('A', end=' ')
            print()


    def calculate_cost(self, cell):

        if cell.cost == -1:
            return 1
        elif cell.cost == -2:
            return 2
        elif cell.cost == -3:
            return float('inf')
        elif cell.cost == -4:
            return 4
        else:
            return 0


    def get_neighbors(self, state):

        x, y = state

        moves = [(x-1, y), (x, y+1), (x-1, y+1)]
        valid_moves = [move for move in moves if 0 <= move[0] < self.size and 0 <= move[1] < self.size ]

        return valid_moves


    def construct_path(self, node):

        path = []

        while node:
            path.append((node.x, node.y))
            node = node.parent

        return path[::-1]


    def heuristic_cost(self, state):
        return abs(self.goal[0] - state[0]) + abs(self.goal[1] - state[1])


    def ucs(self):

        queue = [(0, self.start, [self.start])]
        visited = []

        while queue:

            curr_cost, curr_state, curr_path = heapq.heappop(queue)

            if (curr_state[0], curr_state[1]) == self.goal:
                print(curr_cost)
                return curr_path

            for neighbor in self.get_neighbors(curr_state):

                if self.grid[neighbor[0]][neighbor[1]] != -3 and neighbor not in visited:

                    if neighbor == (curr_state[0]-1, curr_state[1]) or  neighbor == (curr_state[0], curr_state[1]+1):
                        n_cost = 2
                    elif neighbor == (curr_state[0]-1, curr_state[1]+1):
                        n_cost = 3
                    
                    n_cost += curr_cost

                    visited.append(curr_state)
                    heapq.heappush(queue, (n_cost, neighbor, curr_path + [neighbor]))

    
    def ucs2(self):

        queue = [(0, self.start, [self.start])]
        visited = []
        costs_so_far = {}
        costs_so_far[self.start] = 0

        while queue:

            curr_cost, curr_state, curr_path = heapq.heappop(queue)

            if (curr_state[0], curr_state[1]) == self.goal:
                print(curr_cost)
                return curr_path, costs_so_far

            for neighbor in self.get_neighbors(curr_state):

                if self.grid[neighbor[0]][neighbor[1]] != -3 and neighbor not in visited:

                    if neighbor == (curr_state[0]-1, curr_state[1]) or  neighbor == (curr_state[0], curr_state[1]+1):
                        n_cost = 2
                    elif neighbor == (curr_state[0]-1, curr_state[1]+1):
                        n_cost = 3
                    
                    n_cost += curr_cost

                    if neighbor not in costs_so_far:
                        costs_so_far[neighbor] = n_cost
                    elif costs_so_far[neighbor] > n_cost:
                        costs_so_far[neighbor] = n_cost

                    visited.append(curr_state)
                    heapq.heappush(queue, (n_cost, neighbor, curr_path + [neighbor]))


    def a_star_search(self):
        
        queue = [(self.heuristic_cost(self.start), self.start, [self.start])]
        visited = []

        while queue:

            curr_cost, curr_state, curr_path = heapq.heappop(queue)

            if (curr_state[0], curr_state[1]) == self.goal:
                print(curr_cost)
                return curr_path

            for neighbor in self.get_neighbors(curr_state):

                if self.grid[neighbor[0]][neighbor[1]] != -3 and neighbor not in visited:

                    if neighbor == (curr_state[0]-1, curr_state[1]) or neighbor == (curr_state[0], curr_state[1]+1):
                        n_cost = 2
                    elif neighbor == (curr_state[0]-1, curr_state[1]+1):
                        n_cost = 3
                    
                    n_cost += curr_cost + self.heuristic_cost(neighbor) - self.heuristic_cost(curr_state)

                    visited.append(curr_state)
                    heapq.heappush(queue, (n_cost, neighbor, curr_path + [neighbor]))

   
    def a_star_search2(self):

        queue = [(self.heuristic_cost(self.start), self.start, [self.start])]
        visited = []
        costs_so_far = {}
        costs_so_far[self.start] = 0

        while queue:

            curr_cost, curr_state, curr_path = heapq.heappop(queue)

            if (curr_state[0], curr_state[1]) == self.goal:
                print(curr_cost)
                print(costs_so_far[curr_state])
                return curr_path, costs_so_far

            for neighbor in self.get_neighbors(curr_state):

                if self.grid[neighbor[0]][neighbor[1]] != -3 and neighbor not in visited:

                    if neighbor == (curr_state[0]-1, curr_state[1]) or  neighbor == (curr_state[0], curr_state[1]+1):
                        n_cost = 2
                    elif neighbor == (curr_state[0]-1, curr_state[1]+1):
                        n_cost = 3
                    
                    n_cost += costs_so_far[curr_state]

                    if neighbor not in costs_so_far:
                        costs_so_far[neighbor] = n_cost
                    elif costs_so_far[neighbor] > n_cost:
                        costs_so_far[neighbor] = n_cost

                    n_cost += self.heuristic_cost(neighbor)

                    visited.append(curr_state)
                    heapq.heappush(queue, (n_cost, neighbor, curr_path + [neighbor]))


g = grid2(8, a_rate=0.0, start=(7,0), goal=(0, 7))
p = g.ucs()
# p, costs = g.ucs2()
# p = g.a_star_search()
# p, costs = g.a_star_search2()


for path in p:
    if path in costs:
        print(path, costs[path], end=' - ')

print()
g.display(p)

print(p)



    


28
(7, 0) 0 - (6, 0) 2 - (5, 0) 4 - (4, 0) 6 - (3, 0) 8 - (2, 0) 10 - (1, 0) 12 - (0, 0) 14 - (0, 1) 16 - (0, 2) 18 - (0, 3) 20 - (0, 4) 22 - (0, 5) 24 - (0, 6) 26 - (0, 7) 28 - 
0 0 0 0 0 0 0 G 
0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 
S 0 0 0 0 0 0 0 


* * * * * * * G 
* 0 0 0 0 0 0 0 
* 0 0 0 0 0 0 0 
* 0 0 0 0 0 0 0 
* 0 0 0 0 0 0 0 
* 0 0 0 0 0 0 0 
* 0 0 0 0 0 0 0 
S 0 0 0 0 0 0 0 
[(7, 0), (6, 0), (5, 0), (4, 0), (3, 0), (2, 0), (1, 0), (0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7)]
